In [3]:
!pip install pyparsing

In [4]:
import os
import shutil
import random
from PIL import Image
import matplotlib.pyplot as plt
import torch

def display_images(directory: str):
    #Display images from the train dataset 
    # Get the list of image files in the directory
    image_files = [f for f in os.listdir(directory) if f.endswith(('.png', '.jpg', '.jpeg', '.gif'))]

    # Randomly select 'num_images' images
    selected_images = random.sample(image_files, min(6, len(image_files)))

    # Display each selected image
    for image_file in selected_images:
        image_path = os.path.join(directory, image_file)
        img = Image.open(image_path)
    # Display image using Matplotlib
        plt.imshow(img)
        plt.title(image_file)
        plt.axis('off')  # Turn off axis labels
        plt.show()

def create_folder_and_move_files(file_list_path:str, folder_path:str):
    """
    Used to organize data with a given file list and folder name.
    """
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Folder '{folder_path}' created.")

    # Read the list of file names from the text file
    with open(file_list_path, 'r') as file:
        file_names = file.read().splitlines()

    # Move the files to the new folder
    for file_name in file_names:
        source_path = os.path.abspath(file_name)
        destination_path = os.path.join(folder_path, os.path.basename(file_name))

        try:
            shutil.move(source_path, destination_path)
            print(f"File '{file_name}' moved to '{folder_path}'.")
        except FileNotFoundError:
            print(f"File '{file_name}' not found in the current directory.")
        except (PermissionError, shutil.Error) as e:
            print(f"Error moving '{file_name}': {e}")

    print("All files moved successfully.")

def save_model(epochs, model, optimizer, criterion):
    torch.save({
        'epoch': epochs,
        'state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss':criterion
    }, 'outputs/model.pth')

def save_plots(train_acc, valid_acc, train_loss, valid_loss):
    """
    Function to save the loss and accuracy plots to disk.
    """
    # accuracy plots
    plt.figure(figsize=(10, 7))
    plt.plot(
        train_acc, color='green', linestyle='-', 
        label='train accuracy'
    )
    plt.plot(
        valid_acc, color='blue', linestyle='-', 
        label='validataion accuracy'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.savefig('outputs/accuracy.png')
    
    # loss plots
    plt.figure(figsize=(10, 7))
    plt.plot(
        train_loss, color='orange', linestyle='-', 
        label='train loss'
    )
    plt.plot(
        valid_loss, color='red', linestyle='-', 
        label='validataion loss'
    )
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('outputs/loss.png')

ImportError: cannot import name 'Group' from 'pyparsing' (unknown location)

In [7]:
import torch
from torchvision import datasets, transforms

train_dir = 'C:/Users/user/Desktop/LiverCirrhosisClassificationWebApp/liver_disease.v2-release.multiclass/train'
valid_dir = 'C:/Users/user/Desktop/LiverCirrhosisClassificationWebApp/liver_disease.v2-release.multiclass/valid'
BATCH_SIZE = 32
data_transform = transforms.Compose([
        #transforms.RandomSizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Resize((224, 224)),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                             std=[0.229, 0.224, 0.225])
    ])
train_dataset = datasets.ImageFolder(root=train_dir,
                                           transform=data_transform)
train_dataset_loader = torch.utils.data.DataLoader(train_dataset,
                                             batch_size=BATCH_SIZE, shuffle=True,
                                             num_workers=4)

valid_dataset = datasets.ImageFolder(root=valid_dir,
                                           transform=data_transform)
valid_dataset_loader = torch.utils.data.DataLoader(valid_dataset,
                                             batch_size=BATCH_SIZE, shuffle=False,
                                             num_workers=4)

NameError: name '_C' is not defined

In [ ]:
import torch.nn as nn
import torch.functional
import torch.nn.functional as F
class SimpleClassifier(nn.Module):
    def __init__(self):
        super(SimpleClassifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 5)
        self.conv2 = nn.Conv2d(32,64, 5)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.conv4 = nn.Conv2d(128, 256, 5)
        
        self.fc1 = nn.Linear(256, 4)
        self.pool = nn.MaxPool2d(2,2)
        
    def forward(self, x):
            x = self.pool(F.relu(self.conv1(x)))
            x = self.pool(F.relu(self.conv2(x)))
            x = self.pool(F.relu(self.conv3(x)))
            x = self.pool(F.relu(self.conv4(x)))
            
            bs , _ , _, _ = x.shape
            x = F.adaptive_avg_pool2d(x, 1).reshape(bs, -1)
            x = self.fc1(x)
            return x

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm.auto import tqdm

from dataset import train_dataset_loader, valid_dataset_loader
from model import SimpleClassifier
#from utils import save_model, save_plots
#TODO: use argparser to make it a command line thing
lr = 1e-4
EPOCHS = 30
device = ('cuda' if torch.cuda.is_available() else 'cpu')

model = SimpleClassifier().to(device)
print(model)
print(f'Computation on: {device}')

optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

def train(model, datasetloader, optimizer, criterion):
    model.train()
    print('Training...')
    train_running_loss = 0.0
    train_running_correct = 0
    counter = 0
    for i, data in tqdm(enumerate(datasetloader), total=len(datasetloader)):
        image, labels = data
        image = image.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(image)
        loss = criterion(outputs, labels)
        train_running_loss += loss.item()
        _, pred = torch.max(outputs.data, 1)
        train_running_correct += (pred==labels).sum().item()
        loss.backward() #Backprop
        optimizer.step()
        counter +=1
    epoch_loss = train_running_loss/counter
    epoch_acc = 100. *(train_running_correct / len(datasetloader.dataset))
    return epoch_loss, epoch_acc

def validate(model, datasetloader, criterion):
    model.eval()
    print('Validating...')
    val_running_loss = 0.0
    val_running_correct = 0
    counter = 0
    with torch.no_grad():
        for i, data in tqdm(enumerate(datasetloader), total=len(datasetloader)):
            image, labels = data
            image = image.to(device)
            labels = labels.to(device)
            outputs = model(image)
            loss = criterion(outputs, labels)
            val_running_loss += loss.item()
            _, pred = torch.max(outputs.data, 1)
            val_running_correct += (pred == labels).sum().item()
            counter += 1  # Increment counter
    epoch_loss = val_running_loss / counter
    epoch_acc = 100. * (val_running_correct / len(datasetloader.dataset))
    return epoch_loss, epoch_acc

train_loss = []
train_acc = []
valid_loss = []
valid_acc = []

for epoch in range(EPOCHS):
    print(f'Epoch {epoch+1} of {EPOCHS}')
    train_epoch_loss, train_epoch_acc = train(model, train_dataset_loader,
                                               optimizer, criterion)
    valid_epoch_loss, valid_epoch_acc = validate(model, valid_dataset_loader, criterion)
    train_loss.append(train_epoch_loss)
    train_acc.append(train_epoch_acc)
    valid_loss.append(valid_epoch_loss)
    valid_acc.append(valid_epoch_acc)
    print(f'Train Loss: {train_epoch_loss:.3f}, Train Accuracy: {train_epoch_acc:.3f}')
    print(f'Validation Loss: {valid_epoch_loss:.3f}, Validation Accuracy: {valid_epoch_acc:.3f}')
    print("."*50)
